멀티프로세싱에 강하다는 M1 pro 10 core CPU를 산 기념으로 어떻게 하면 모든 코어를 불타게 만들 수 있을까를 찾아보고 싶어졌다. 과연 10개의 코어를 모두 한꺼번에 돌리면 작업 수행 시간을 어디까지 줄일 수 있을까?

&nbsp;

시작하기 전에 내장 os 모듈을 이용해서 CPU core 개수를 확인해보자. 10개의 core가 잘 확인된다.

In [1]:
import os
os.cpu_count()

10

---

### Mutiprocessing module

가장 먼저 파이썬에 내장된 multiprocessing 모듈을 사용해보자. 
목표는 0.01초가 걸리는 일을 1000번 반복하는 것이고 한 개의 프로세스로 시행한다면 10초 이상 걸린다.

In [14]:
import time
def simple_work(a):
    time.sleep(0.01)

In [15]:
%%time
for a in range(1000): 
    simple_work(a)

CPU times: user 16.8 ms, sys: 9.02 ms, total: 25.8 ms
Wall time: 12 s



&nbsp;

multiprocessing에는 Process 클래스와 Pool 클래스가 있다. 
먼저 Process 클래스에서는 process에서 진행할 일들을 지정해 줘야 한다. 분리된 lst를 시행시킬 worker 함수를 만들고 python 파일로 저장하자.

In [ ]:
# test_work.py
import time
def simple_work(a):
    time.sleep(0.01)

def simple_worker(divided_lst):
    for a in divided_lst:
        simple_work(a)

> multiprocessing은 jupyter에서 그대로 사용 하면 AttributeError가 발생한다.
> python 파일에서 시행하거나 시행할 함수를 .py 파일로 따로 만든 후 module로 불러와야 한다.


&nbsp;

5개의 프로세스로 나눴더니 수행 시간이 5분의 1로 줄어들었다.

In [16]:
from multiprocessing import Process
from test_work import simple_worker

lst1 = list(range(1000))

In [17]:
%%time
processes = []
for i in range(5):
    p = Process(target=simple_worker, args=(lst1[i*200:(i+1)*200],))
    p.start()
    processes.append(p)

for process in processes[:1]:
    process.join()


CPU times: user 6.32 ms, sys: 35.8 ms, total: 42.1 ms
Wall time: 2.46 s



&nbsp;


---

Pool 클래스는 process를 지정해서 나누는 수고를 덜어줘 더욱 간단하게 사용할 수 있다.

In [18]:
from multiprocessing import Pool
from test_work import simple_work

In [19]:
%%time
with Pool(processes=5) as pool:
    pool.map(simple_work, lst1)


CPU times: user 13.4 ms, sys: 37.1 ms, total: 50.4 ms
Wall time: 2.52 s



&nbsp;


---

이제 multiprocessing을 알게 되었으니 신나게 쓰려고 하다보면 이상함을 느끼게 된다. multiprocessing이 그냥 계산하는 것 보다 더 오래걸리는 것이다. 심지어 process 수를 늘렸더니 수행 시간이 더 늘어난다.

In [20]:
import math
lst2 = list(range(1000000))

In [21]:
%%time
result = [math.sqrt(a) for a in lst2]

CPU times: user 97.8 ms, sys: 15.2 ms, total: 113 ms
Wall time: 112 ms


In [25]:
%%time
with Pool(processes=5) as pool:
    result = pool.map(math.sqrt, lst2)

CPU times: user 69.5 ms, sys: 74.6 ms, total: 144 ms
Wall time: 178 ms


In [24]:
%%time
with Pool(processes=9) as pool:
    result = pool.map(math.sqrt, lst2)

CPU times: user 77.4 ms, sys: 104 ms, total: 181 ms
Wall time: 189 ms


원인을 찾아 보았더니 multiprocessing에서 객체 전달 시스템의 문제라고 한다. multiprocess는 process spawning을 통해 자식 프로세스를 생성하고, 이 과정에서 데이터를 pickle을 사용해 직렬화 한 후 복사본을 만든다고 한다. 따라서 데이터가 클 수록, 프로세스가 늘어날 수록 걸리는 시간도 오래 걸리고 메모리 사용량도 늘어나게 된다. (10 core M1 pro를 산 이유가 없어지는 것이 아닌가!!!)

이 문제점을 해결할 방법을 찾다가 ray를 발견하게 되었다.

&nbsp;

---

### Ray
> ray는 multiprocessing에서 발생한 큰 오버헤드 문제를 해결하기 위해 zero-copy 직렬화를 수행하는 Apache Arrow를 사용한다고 한다. 자세한 설명은 reference에서 확인하자.  ( 
>[[reference1]](https://riiidtechblog.medium.com/ray-%ED%99%95%EC%9E%A5-%EA%B0%80%EB%8A%A5%ED%95%9C-%EA%B3%A0%EC%84%B1%EB%8A%A5-%EB%B6%84%EC%82%B0-%EB%B3%91%EB%A0%AC-machine-learning-%ED%94%84%EB%A0%88%EC%9E%84%EC%9B%8C%ED%81%AC-f17f9c9cbef3)
[[reference2]](https://velog.io/@otzslayer/Ray%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%B4-Python-%EB%B3%91%EB%A0%AC-%EC%B2%98%EB%A6%AC-%EC%89%BD%EA%B2%8C-%ED%95%98%EA%B8%B0))


&nbsp;

ray를 사용하기 위해서는 4가지 과정이 필요하다.

(1) remote class를 사용하여 actor만들기
- 원하는 function을 ```@ray.remote``` 데코레이터로 감싼다.
    
(2) ray 시작하기 (ray.init)

(3) tasks(remote functions) 만들기
- 1번에 만든 function을 .remote()로 호출하여 비동기적으로 시행할 remote function들을 만든다.

(4) ```ray.get```을 이용하여 result 가져오기

이 과정이 끝나면 ```ray.shutdown()```을 꼭 실행해야 다음에 다시 ray를 불러올 수 있다.


&nbsp;


연산량에 비해 작업 반복 횟수가 많은 위의 예제는 ray에 적절하지 않다. 반복 횟수 만큼 remote function object를 만들기 때문에 그에 비례해 시간이 늘어난다. ray의 사용이 적절한 예는 아래와 같다.

In [50]:
import numpy as np

def make_rand_arr(num):
    return np.random.rand(1000,1000)

def multiple_arr(arr1,arr2):
    return arr1@arr2

In [62]:
%%time
result = []

for i in range(10):
    arr1 = make_rand_arr(i)
    arr2 = make_rand_arr(i)
    result.append(multiple_arr(arr1,arr2))

CPU times: user 1.55 s, sys: 264 ms, total: 1.81 s
Wall time: 294 ms



&nbsp;


In [64]:
import ray

@ray.remote
def make_rand_arr(num):
    return np.random.rand(1000,1000)

@ray.remote
def multiple_arr(arr1,arr2):
    return arr1@arr2
    
ray.shutdown()
ray.init(num_cpus = 5)


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:29059',
 'object_store_address': '/tmp/ray/session_2022-01-16_12-39-00_701759_2973/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-16_12-39-00_701759_2973/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-01-16_12-39-00_701759_2973',
 'metrics_export_port': 65485,
 'node_id': '12a2284120523d8a9f5dccf38be1f74b2001e524243d00d238034a04'}

In [65]:
%%time
arr1_id = [make_rand_arr.remote(i) for i in range(10)]
arr2_id = [make_rand_arr.remote(i) for i in range(10)]
multi_id = [multiple_arr.remote(arr1,arr2) for arr1, arr2 in tuple(zip(arr1_id, arr2_id))]

CPU times: user 9.65 ms, sys: 4.72 ms, total: 14.4 ms
Wall time: 9.82 ms


In [66]:
%%time
result = ray.get(multi_id)


CPU times: user 1.71 ms, sys: 826 µs, total: 2.54 ms
Wall time: 1.43 ms


병렬처리 하지 않았을 시 수행 시간은 300ms, ray 적용 시 10ms로 큰 차이를 보인다.

&nbsp;

---

multiprocessing과는 더 큰 차이를 보인다.

In [31]:
from multiprocessing import Pool
from test_work2 import make_rand_arr, multiple_arr

In [32]:
%%time
with Pool(processes=5) as pool:
    arr1 = pool.map(make_rand_arr, range(10))
    arr2 = pool.map(make_rand_arr, range(10))
    result = pool.starmap(multiple_arr, tuple(zip(arr1,arr2)))

CPU times: user 208 ms, sys: 446 ms, total: 654 ms
Wall time: 1.06 s
